# Site-specific security and privacy policies

NVIDIA FLARE offers a set of security mechanisms to control user access for different operations based on their roles. These controls are not limited to centralized server control, but also exist at each site. Such site-specific level of security is done via security policies which we call federated policies. 

[FLARE's Security documentation](https://nvflare.readthedocs.io/en/main/user_guide/security/identity_security.html) has details regarding
* Centralized vs. Federated Authorization 
* Policy Configuration 
* Roles and Rights
* Controls and Conditions
* Command Categories
* Policy Evaluation
* Command Authorization Process
* Job Submission authorization process
* Job Management Commands authorization 

This section we are going dive into the usage of these policies via few examples, but first a refresher of terminology used in the examples. 

n the privacy policy, we notice that different scopes can be defined. The scope-specified policy is enforced by the FLARE's filter mechanism. For each non-public scope, one can define different filters to enforce the desired behavior. For example, the organization administrator can implement filters to prevent certain types of data leakage from being accidentally exposed by data scientists.
_____________


Refresher: Command Category and Control Notations
----------
Before we go to the Test Cases, let's refresh some concepts and notatations

**Command Category**

| permission       | command         |
|------------------|-----------------|
| MANAGE_JOB       | ABORT           |
| MANAGE_JOB       | ABORT_JOB       |
| MANAGE_JOB       | START_APP       |
| MANAGE_JOB       | DELETE_JOB      |
| MANAGE_JOB       | DELETE_WORKSPACE|
| VIEW             | CHECK_STATUS    |
| VIEW             | SHOW_STATS      |
| VIEW             | RESET_ERRORS    |
| VIEW             | SHOW_ERRORS     |
| VIEW             | LIST_JOBS       |
| OPERATE          | SYS_INFO        |
| OPERATE          | RESTART         |
| OPERATE          | SHUTDOWN        |
| OPERATE          | REMOVE_CLIENT   |
| OPERATE          | SET_TIMEOUT     |
| OPERATE          | CALL            |
| SHELL_COMMANDS   | SHELL_CAT       |
| SHELL_COMMANDS   | SHELL_GREP      |
| SHELL_COMMANDS   | SHELL_HEAD      |
| SHELL_COMMANDS   | SHELL_LS        |
| SHELL_COMMANDS   | SHELL_PWD       |
| SHELL_COMMANDS   | SHELL_TAIL      |



**Notation and Condition**

| Notation       | Condition                                      | Examples            |
|----------------|------------------------------------------------|---------------------|
| o:site         | The user belongs to the site’s organization    |                     |
| n:submitter    | The user is the job submitter                  |                     |
| o:submitter    | The user and the job submitter belong to the same org |             |
| n:<person_name>| The user is a specified person                 | n:john@nvidia.com   |
| o:<org_name>   | The user is in a specified org                 | o:nvidia            |

The words “site” and “submitter” are reserved.

In addition, two words are used for extreme conditions:

* Any user is allowed: any
* No user is allowed: none

control is a set of one or more conditions that is specified in the permission matrix. Conditions specify relationships among the subject user, the site, and the job submitter. The following are supported relationships:

* The user belongs to the site’s organization (user org = site org)
* The user is the job submitter (user name = submitter name)
* The user and the job submitter are in the same org (user org = submitter org)
* The user is a specified person (user name = specified name)
* The user is in a specified org (user org = specified org)
* Keep in mind that the relationship is always relative to the subject user - we check to see whether the user’s name or org has the right relationship with the site or job submitter.




---- 
## Overview


## Overview

Now we are ready to discuss examples for federated site policy. The purpose of these examples is to demonstrate the following features of NVIDIA FLARE:

1. Show secure FLARE console and authentication
2. Demonstrate local authorization policy 
3. Demonstrate local privacy policy

## Participants

Site
____
* `server`: NVIDIA FLARE server owned by org a
* `site_a`: Client owned by a.org with a customized authorization policy, which only allows users from the same org to submit jobs
* `site_b`: Client owned by b.org with a customized privacy policy. The policy defines two scopes: `public` and `private`. A custom filter is applied to `private`

Users
_____
*```super@a.org```: Super user with role `project_admin` who can do everything
*```admin@a.org```: Admin for a.org with role `org_admin`
*```trainer@a.org```: Lead trainer for a.org with role `lead`
*```trainer@b.org```: Lead trainer for b.org with role `lead`
*```user@b.org```: Regular user for b.org with role `member`

Jobs
____
All the jobs run the same app but have different scopes defined in `meta.json`.

* job1: Scope is `public`. No filters.
* job2: Scope is `test`. Test filters are applied to data and result.
* job3: Scope is `private`. PercentilePrivacy filter is applied to result.
* job4: It has no scope defined.
* job5: It defines an non-existent scope `foo`



### Test Cases
----------




Test Cases
----------

Authorization
_____________
We will demo some authorization behaviors.

Since authorization decision is determined using each site's authorization.json and each admin user's role,
we just use `job1` in all the following tests.

| User           | Command                                      | Expected behavior                                                                 |
|----------------|----------------------------------------------|-----------------------------------------------------------------------------------|
| trainer@a.org  | submit_job  /tmp/nvflare/jobs/workdir/job_1                        | Job deployed and started on all sites                                             |
| trainer@a.org  | clone_job [the job ID that we previous submitted] | Job deployed and started on all sites                                             |
| trainer@b.org  | clone_job [the job ID that we previous submitted] | Rejected because submitter is in a different org                                  |
| admin@a.org    | submit_job /tmp/nvflare/jobs/workdir/job_1                      | Rejected because role "org_admin" is not allowed to submit jobs                   |
| trainer@b.org  | submit_job /tmp/nvflare/jobs/workdir/job_1                        | site_a rejected the job because the submitter is in a different org, while site_b accepted the job so the job will still run since in meta.json we specify min_clients as 1 |

Privacy
_______
site_a has no privacy policy defined.
So we will test the following cases on site_b.

In each job's meta.json we specified their "scope" and in site's privacy.json file each site will define its own
privacy filters to apply for that scope.

Note that default jobs are treated in "public" scope.

Let's just use user trainer@b.org for the following tests.

| Job  | Expected behavior |
|------|--------------------|
| job1 | Job deployed with no filters |
| job2 | Job deployed with TestFilter applied |
| job3 | Job deployed with PercentilePrivacy filter applied to the result |
| job4 | Job deployed using default scope `public` |
| job5 | Job rejected by site_b because `foo` doesn't exist |


Setup FL System, Site Policies
----------

* Prepare POC with given project.yml file

In [3]:
! echo y | nvflare poc prepare -i code/federated-policies/project.yml

prepare poc at /tmp/nvflare/poc with code/federated-policies/project.yml
This will delete poc workspace directory: '/tmp/nvflare/poc' and create a new one. Is it OK to proceed? (y/N) provision at /tmp/nvflare/poc for 2 clients with code/federated-policies/project.yml
Generated results can be found under /tmp/nvflare/poc/fed_policy/prod_00. 


* Setup polices for different sites 

In [7]:
Workspace =  "/tmp/nvflare/poc/fed_policy/prod_00"

In [ ]:

! cp -r code/federated-policies/policies/site_a/* {Workspace}/site_a/local
! cp -r code/federated-policies/policies/site_b/* {Workspace}/site_b/local

We can take a look at the policies for site_a


In [ ]:
!cat {Workspace}/site_a/local/authorization.json

##### Site-a Security Policy

the user "org_admin"

| Capability       | Permission Scope |
|------------------|------------------|
| submit_job       | none             |
| clone_job        | none             |
| manage_job       | o:submitter      |
| download_job     | o:submitter      |
| view             | any              |
| operate          | o:site           |
| shell_commands   | o:site           |
| byoc             | none             |


These table essentially set the policy for the Organization Admin "org_admin" 
* can not submit job 
* can not clone job
* can manage job (such as abort job) for the jobs that submitted by the "job sbumitter" from the same organization. The "job submitter" is the user who has the submit_job permission 
* can download the jobs submitted by the same organziation 
* can view any jobs
* can perform shell commands ( see job category for details) from the site of the same organization 
* can not have byoc (bring your own code): customized code in the "custom" directory. 

Similarly we have other user role policies


For the **lead** user

| Capability       | Permission Scope |
|------------------|------------------|
| submit_job       | o:site           |
| clone_job        | n:submitter      |
| manage_job       | n:submitter      |
| download_job     | n:submitter      |
| view             | any              |
| operate          | o:site           |
| shell_commands   | o:site           |
| byoc             | any              |

The user "lead" is a "submitter" that can but only can submit to its organizational own site (considering one organization has many sites)



For the **member** user

| Capability | Permission Scope |
|------------|------------------|
| view       | any              |

    

We can take a look at the policies for 

##### Site_b privacy policy

In [ ]:
!cat {Workspace}/site_b/local/privacy.json

| scope   | default_scope | task_data_filters                       | task_result_filters                                                |
|---------|----------------|-----------------------------------------|--------------------------------------------------------------------|
| public  | public         |                                         |                                                                    |
| test    | public         | test_filter.TestFilter                  | test_filter.TestFilter                                             |
| private | public         | test_filter.TestFilter                  | nvflare.app_common.filters.percentile_privacy.PercentilePrivacy    |


In the privacy policy, we noticed that there are different scopes can be defined. The scope-specified policy is enforced the the FLARE's filter mechanism. For each non-public scope, one can define the different filters to enforce the desired behavior.  For example, the organization administrator can instrument filter to prevent certain type of data leakage accidentally exposed by data sicentists

The filters are look like this: 


In [ ]:
!tree {Workspace}/site_b/local

Let's prepare the jobs. we have five different jobs each have different groups. 

##### Create Job Configs

In [4]:
%cd code/federated-policies/jobs

! python fl_job.py

%cd -

/home/chester/projects/NVFlare/examples/tutorials/self-paced-training/part-3_security_and_privacy/chapter-6_Security_in_federated_compute_system/06.3_site_security_privacy_policy/code/federated-policies/jobs


/home/chester/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


job-config is at  /tmp/nvflare/jobs/workdir/job_1
job-config is at  /tmp/nvflare/jobs/workdir/job_2
job-config is at  /tmp/nvflare/jobs/workdir/job_3
job-config is at  /tmp/nvflare/jobs/workdir/job_4
job-config is at  /tmp/nvflare/jobs/workdir/job_5
/home/chester/projects/NVFlare/examples/tutorials/self-paced-training/part-3_security_and_privacy/chapter-6_Security_in_federated_compute_system/06.3_site_security_privacy_policy


In [ ]:
!cat   /tmp/nvflare/jobs/workdir/job_3/meta.json

### Start FL System and Run Jobs

Start POC with 

```nvflare poc start 
